In [1]:
import numpy as np
import pandas as pd
from os import listdir
import seaborn as sns
import functools as ft
import matplotlib.pyplot as plt

In [2]:
is_folder = '/content/drive/MyDrive/DS_Capstone/data/nasdaq/quarterly/income_statement'
bs_folder = '/content/drive/MyDrive/DS_Capstone/data/nasdaq/quarterly/balance_sheet'
cf_folder = '/content/drive/MyDrive/DS_Capstone/data/nasdaq/quarterly/cash_flow'
macro_file = '/content/drive/MyDrive/DS_Capstone/data/macro/macro.csv'
dfs = {
    'is': [pd.read_csv(is_folder + '/' + csv) for csv in listdir(is_folder)],
    'bs': [pd.read_csv(bs_folder + '/' + csv) for csv in listdir(bs_folder)],
    'cf': [pd.read_csv(cf_folder + '/' + csv) for csv in listdir(cf_folder)]
       }
df_all_by_type = {'is': pd.concat(dfs['is']), 'bs': pd.concat(dfs['bs']), 'cf': pd.concat(dfs['cf'])}
df_all = ft.reduce(lambda left, right: pd.merge(left, right, on=['symbol', 'date']), list(df_all_by_type.values()))

df_macro = pd.read_csv(macro_file)

In [3]:
df_all['date'].astype('datetime64[ns]')
df_all.sort_values(by=['symbol', 'date'], inplace=True)
df_all = df_all[df_all['symbol'].map(df_all['symbol'].value_counts()) >= 20]
df_all.shape

(171560, 128)

# Preprocessing

In [4]:
# filter out features containing lots of NAs based on cutoff

cutoff = 0.05
col_na = np.mean(df_all.select_dtypes(include=['float64']).isna())
f_left = col_na[col_na <= cutoff].index.to_list()
f_left.remove('netIncome_y')

# find and delete symbols contributing NAs remains

df_tmp = df_all[['date', 'symbol'] + f_left]

symbols_to_drop = df_tmp[df_tmp.isnull().any(axis=1)]['symbol'].unique()

df_raw = df_tmp[~df_tmp['symbol'].isin(symbols_to_drop)]
df_raw.shape

(16368, 17)

In [ ]:
# check multicollinearity

sns.heatmap(df_raw[f_left].corr(), cmap='GnBu', annot=True)
plt.rcParams['figure.figsize'] = (30, 10)
plt.show()

In [ ]:
look_back = 6
targets = [
 'ebit',
 'totalOperatingExpenses',
 'totalLiab',
 'totalStockholderEquity',
 'commonStockSharesOutstanding',
 'otherCashflowsFromFinancingActivities',
 'capitalExpenditures']

features = targets + ['GDP', 'UNEMP', 'CPI', 'PPI', 'INDO']
df_raw_mac = df_raw.merge(df_macro, on=['date'], how='left')
# df_raw = pd.read_csv('data.csv')

# Exploratory Analysis

In [ ]:
df_raw_scaled = pd.DataFrame(np.hstack((np.array(df_raw[['date', 'symbol']]), 
                   np.vstack([MinMaxScaler().fit_transform(y.drop(columns=['symbol', 'date'])) for _, y in df_raw.groupby('symbol')])
                   )), columns=df_raw.columns)
df_raw_scaled['quarter'] = df_raw_scaled.date.str[-5:-3].astype(int)%12 // 3 + 1

fig, axes = plt.subplots(len(targets), figsize=(10, 15), sharex=True)
for i, f in enumerate(targets): sns.boxplot(data=df_raw_scaled, x='quarter', y=f, ax=axes[i])
plt.show()

# Utils functions

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from numpy.lib.stride_tricks import sliding_window_view
import seaborn as sns
import matplotlib.pyplot as plt


# convert sequences into samples
def df2samples(df, look_back, features, targets):
  train_X, train_Y, test_X, test_Y = [], [], [], []
  for _, df_symbol in df.groupby('symbol'):
    
    df_symbol = df_symbol.drop(columns=['symbol', 'date'])
    
    scaler = MinMaxScaler()
    scaler.fit_transform(df_symbol.iloc[:-2])
    df_scaled = pd.DataFrame(scaler.transform(df_symbol), columns=df_symbol.columns)
    
    all_Y = df_scaled[targets].to_numpy()[look_back:]
    all_X = sliding_window_view(df_scaled[features].to_numpy()[:-1], (look_back, len(features))).squeeze()

    train_X.append(all_X[:-1]), test_X.append(all_X[-1].reshape((1, look_back, len(features))))
    train_Y.append(all_Y[:-1]), test_Y.append(all_Y[-1])
  return np.vstack(train_X), np.vstack(train_Y), np.vstack(test_X), np.vstack(test_Y)


# plot grid search result
def gs_heatmap(values, x, y, x_label='LSTM units', y_label='look_back'):
  values = np.array(values).reshape((len(y), len(x)))
  fig = sns.heatmap(values, cmap='rocket_r', square=True, linewidth=.2, xticklabels=x, yticklabels=y)
  fig.set_ylabel(y_label)
  fig.set_xlabel(x_label)
  fig.set_title('Loss by grid search')
  cbar = fig.collections[0].colorbar
  cbar.set_ticks([cbar.get_ticks()[-1], cbar.get_ticks()[0]])
  cbar.set_ticklabels(['High', 'Low'])

  plt.tight_layout()
  plt.show()


# smape for evaluation
def SMAPE(y_pred, y_act): 
    n = np.abs((y_act - y_pred))
    d = np.abs(y_act) + np.abs(y_pred)
    return 200 * np.mean(np.where(d == 0, 0, n / d), 0)

# smape for validation
def smape(y_true, y_pred):
    import keras.backend as K
    return 200 * K.mean(K.abs(y_pred - y_true) / ((K.abs(y_true) + K.abs(y_pred))), axis=-1)

# ARIMA - Baseline

In [ ]:
!pip install pmdarima -q
from pmdarima.arima import auto_arima


def ARIMA(df, features):
    
    test_Y, pred_Y = [], []
    for _, df_symbol in df.groupby('symbol'):
        df_symbol = df_symbol[features]
        scaler = MinMaxScaler()
        scaler.fit_transform(df_symbol.iloc[:-1])
        all_X = scaler.transform(df_symbol)
        pred_Y.append([auto_arima(all_X[:-1, i]).predict(n_periods=1)[0] for i in range(len(features))])
        test_Y.append(all_X[-1])

        
    pred_Y, test_Y = np.stack(pred_Y), np.stack(test_Y)
    return SMAPE(pred_Y, test_Y)

ARIMA(df_raw, targets)

# Multivariate LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint, Callback
import tensorflow as tf
from datetime import datetime

class LSTM_m2m(object):

    def __init__(self, units, look_back, n_features, n_targets):


        # Get model hyperparameters
        self.units = units
        self.look_back = look_back
        self.n_features = n_features
        self.n_targets = n_targets

        # Get directories name
        self.log_dir = 'log'
        self.checkpoint_dir = 'checkpoint'
        self.dropout = 0.25


    def build(self):
        model = Sequential()
        model.add(LSTM(self.units, input_shape=(self.look_back, self.n_features)))
        # model.add(LSTM(units, input_shape=(self.look_back, self.n_features), return_sequences=True))
        # model.add(LSTM(64))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(self.n_targets))
        
        return model

    def restore(self, filepath):

        # Load the architecture
        self.best_model = load_model(filepath, custom_objects={'smape': smape})
        self.best_model.compile(optimizer='adam', loss=['mse'], metrics=[smape])


    def train(self, X_train, y_train, epochs=200, batch_size=64, verbose=1):

        self.model = self.build()
        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=['mse'], metrics=[smape])

        # Stop training if error does not improve within 25 iterations
        early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

        # Save the best model ... with minimal error
        filepath = self.checkpoint_dir+'/Transformer.best'+datetime.now().strftime('%d%m%Y_%H:%M:%S')+'.hdf5'
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=verbose, save_best_only=True, mode='min')

        callback_history = self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                             validation_split=0.2, verbose=verbose,
                             callbacks=[early_stopping_monitor, checkpoint])
        return callback_history


    def evaluate(self, X_test, y_test):

        y_pred = self.model.predict(X_test)
        loss, _ = self.model.evaluate(X_test, y_test, verbose=0) 

        return loss, SMAPE(y_pred, y_test)

In [ ]:
# LSTM trained on targts

lstm_result = []

for look_back in range(4, 17, 2):
  train_X, train_Y, test_X, test_Y = df2samples(df_raw, look_back, targets, targets)
  for units in np.linspace(64, 128, 6, dtype=int):
    lstm = LSTM_m2m(units, look_back, len(targets), len(targets))
    lstm.train(train_X, train_Y, batch_size=256, verbose=0)
    lstm_result.append(lstm.evaluate(test_X, test_Y))

gs_loss = [result[0] for result in lstm_result]
gs_heatmap(gs_loss, x=np.linspace(64, 128, 6, dtype=int), y=range(4, 17, 2))
print('optimal LSTM on test: ', lstm_result[np.argmin(gs_loss)][1])

In [ ]:
# LSTM trained on features including macro indicators

lstm_m_result = []

for look_back in range(4, 17, 2):
  train_X, train_Y, test_X, test_Y = df2samples(df_raw_mac, look_back, features, targets)
  for units in np.linspace(64, 128, 6, dtype=int):
    lstm = LSTM_m2m(units, look_back, len(features), len(targets))
    lstm.train(train_X, train_Y, batch_size=256, verbose=0)
    lstm_m_result.append(lstm.evaluate(test_X, test_Y))

gs_m_loss = [result[0] for result in lstm_m_result]
gs_heatmap(gs_m_loss, x=np.linspace(64, 128, 6, dtype=int), y=range(4, 17, 2))
print('optimal LSTM with macro on test: ', lstm_m_result[np.argmin(gs_m_loss)][1])

# Transformer

In [ ]:
!pip install keras-tuner --upgrade -q


from keras import Input, Model
from keras.layers import LayerNormalization, MultiHeadAttention, Dropout, Conv1D, GlobalAveragePooling1D, Dense
from keras.models import Model, load_model, Sequential
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint, Callback

from keras_tuner.tuners import RandomSearch


class Transformer(object):

    def __init__(self, look_back, n_features, n_targets):

        with open('parameters.json') as f:
            parameters = json.load(f)


        # Get model hyperparameters
        self.look_back = look_back
        self.n_features = n_features
        self.n_targets = n_targets
        # Get directories name
        self.log_dir = 'log'
        self.checkpoint_dir = 'checkpoint'

        self.head_size=256
        self.num_heads=10
        self.ff_dim=4
        self.num_transformer_blocks=6
        self.mlp_units=[128]
        self.mlp_dropout=0.1
        self.dropout=0.1


    def transformer_encoder(self, inputs):

        # Normalization and Attention
        x = LayerNormalization(epsilon=1e-6)(inputs)
        x = MultiHeadAttention(key_dim=self.head_size, num_heads=self.num_heads, dropout=self.dropout)(x, x)
        x = Dropout(self.dropout)(x)

        res = x + inputs

        # Feed Forward Part
        x = LayerNormalization(epsilon=1e-6)(res)
        x = Conv1D(filters=self.ff_dim, kernel_size=1, activation='relu')(x)
        x = Dropout(self.dropout)(x)
        x = Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
        return x + res


    def build(self):

        inputs = Input(shape=(self.look_back, self.n_features))
        x = inputs
        for _ in range(self.num_transformer_blocks): x = self.transformer_encoder(x)

        x = GlobalAveragePooling1D(data_format='channels_first')(x)
        for dim in self.mlp_units:
            x = Dense(dim, activation='relu')(x)
            x = Dropout(self.mlp_dropout)(x)

        # output layer
        outputs = Dense(self.n_targets)(x)

        return Model(inputs, outputs)

    def restore(self, filepath):

        # Load the architecture
        self.best_model = load_model(filepath, custom_objects={'smape': smape})
        self.best_model.compile(optimizer='adam', loss=['mse'], metrics=[smape])


    def train(self, X_train, y_train, epochs=200, batch_size=256, verbose=1):

        self.model = self.build()
        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=['mse'], metrics=[smape])


        # Stop training if error does not improve within 25 iterations
        early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

        # Save the best model ... with minimal error
        filepath = self.checkpoint_dir+'/Transformer.best'+datetime.now().strftime('%d%m%Y_%H:%M:%S')+'.hdf5'
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=verbose, save_best_only=True, mode='min')

        callback_history = self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                             validation_split=0.2, verbose=verbose,
                             callbacks=[early_stopping_monitor, checkpoint])



    def evaluate(self, X_test, y_test):

        y_pred = self.model.predict(X_test)
        loss, _ = self.model.evaluate(X_test, y_test, verbose=0) 

        return loss, SMAPE(y_pred, y_test)

In [ ]:
# Transformer trained on targts
tr_result = []

for look_back in range(4, 17, 2):
  train_X, train_Y, test_X, test_Y = df2samples(df_raw, look_back, targets, targets)
  tr = Transformer(look_back, len(targets), len(targets))
  tr.train(train_X, train_Y, verbose=0)
  tr_result.append(tr.evaluate(test_X, test_Y))

gs_loss = [result[0] for result in tr_result]
print('optimal Transformer on test: ', tr_result[np.argmin(gs_loss)][1])
    
plt.plot(range(4, 17, 2), gs_loss)
plt.xlabel('look_back')
plt.ylabel('MSE')
plt.title('Transformer Loss')
plt.show()

In [ ]:
# Transformer trained on features including macro indicators

tr_m_result = []

for look_back in range(4, 17, 2):
  train_X, train_Y, test_X, test_Y = df2samples(df_raw, look_back, features, targets)
  tr = Transformer(look_back, len(features), len(targets))
  tr.train(train_X, train_Y, verbose=0)
  tr_m_result.append(tr.evaluate(test_X, test_Y))

gs_loss = [result[0] for result in tr_m_result]
print('optimal Transformer with macro on test: ', tr_m_result[np.argmin(gs_loss)][1])
    
plt.plot(range(4, 17, 2), gs_loss)
plt.xlabel('look_back')
plt.ylabel('MSE')
plt.title('Transformer Loss')
plt.show()